In [4]:
import connection
import pathlib

# location of PAT
pat = pathlib.Path('/Users/aniket/ws/tokens/prod.token')

# legacy JDBC driver location
driver = pathlib.Path('/Users/aniket/downloads/dremio-jdbc-driver-24.3.2-202401241821100032-d2d8a497.jar')


## Connect using Legacy JDBC

In [5]:
# project ID for the project to connect to
project_id = 'f03ab262-af2f-46bc-a719-2dbc22a06453'
params = connection.JdbcConnectionParams(connection.Region.NA, pat.read_text(), connection.JdbcConnectionParams.TokenType.PAT,
                                         project_id=project_id, driver_jar=driver)
jconn = params.connect()

In [6]:
# work with sample data
with jconn.cursor() as cur:
    cur.execute(
        'select pickup_datetime, passenger_count, total_amount from Samples."samples.dremio.com"."NYC-taxi-trips-iceberg" limit 10')
    while row := cur.fetchone():
        print(row)

('2013-02-10 21:59:00', 2, 9.5)
('2013-02-10 22:00:00', 6, 7.0)
('2013-02-10 21:57:00', 1, 8.62)
('2013-02-10 21:39:00', 1, 57.3)
('2013-02-08 09:44:00', 1, 15.45)
('2013-02-10 22:01:00', 1, 8.3)
('2013-02-10 21:47:00', 1, 29.0)
('2013-02-08 09:47:00', 1, 12.5)
('2013-02-08 09:53:00', 1, 10.7)
('2013-02-08 09:54:00', 5, 5.0)


## Connect using ADBC

This has the additional benefit of integrating directly with pandas

In [15]:
import pandas as pd
import warnings
warnings.filterwarnings(action='ignore', category=UserWarning)
import importlib
importlib.reload(connection)

params = connection.AdbcConnectionParams(connection.Region.NA, pat.read_text(), project_id=project_id)
conn = params.connect()

k = adbc.flight.sql.rpc.call_header.Cookie, v = project_id=f03ab262-af2f-46bc-a719-2dbc22a06453


In [16]:
pd.read_sql(
    'select * from Samples."samples.dremio.com"."NYC-taxi-trips-iceberg" limit 10', conn)

,pickup_datetime,passenger_count,trip_distance_mi,fare_amount,tip_amount,total_amount
0,2013-02-10 21:59:00,2,1.61,7.0,1.50,9.50
1,2013-02-10 22:00:00,6,0.98,6.0,0.00,7.00
2,2013-02-10 21:57:00,1,1.09,6.0,1.62,8.62
3,2013-02-10 21:39:00,1,16.84,52.0,0.00,57.30
4,2013-02-08 09:44:00,1,1.95,11.5,3.45,15.45
5,2013-02-10 22:01:00,1,0.87,5.5,1.80,8.30
6,2013-02-10 21:47:00,1,9.92,28.0,0.00,29.00
7,2013-02-08 09:47:00,1,1.42,10.0,2.00,12.50
8,2013-02-08 09:53:00,1,1.51,8.5,1.70,10.70
9,2013-02-08 09:54:00,5,0.44,4.5,0.00,5.00


In [14]:
print({
	"name": "DatabaseError",
	"message": "Execution failed on sql 'select * from Samples.\"samples.dremio.com\".\"NYC-taxi-trips-iceberg\" limit 10': UNKNOWN: [FlightSQL] [FlightSQL]  (Unknown; Prepare)",
	"stack": "\u001b[0;31m---------------------------------------------------------------------------\u001b[0m\n\u001b[0;31mOperationalError\u001b[0m                          Traceback (most recent call last)\nFile \u001b[0;32m/opt/homebrew/lib/python3.11/site-packages/pandas/io/sql.py:2266\u001b[0m, in \u001b[0;36mSQLiteDatabase.execute\u001b[0;34m(self, sql, params)\u001b[0m\n\u001b[1;32m   2265\u001b[0m \u001b[38;5;28;01mtry\u001b[39;00m:\n\u001b[0;32m-> 2266\u001b[0m     \u001b[43mcur\u001b[49m\u001b[38;5;241;43m.\u001b[39;49m\u001b[43mexecute\u001b[49m\u001b[43m(\u001b[49m\u001b[43msql\u001b[49m\u001b[43m,\u001b[49m\u001b[43m \u001b[49m\u001b[38;5;241;43m*\u001b[39;49m\u001b[43margs\u001b[49m\u001b[43m)\u001b[49m\n\u001b[1;32m   2267\u001b[0m     \u001b[38;5;28;01mreturn\u001b[39;00m cur\n\nFile \u001b[0;32m/opt/homebrew/lib/python3.11/site-packages/adbc_driver_manager/dbapi.py:621\u001b[0m, in \u001b[0;36mCursor.execute\u001b[0;34m(self, operation, parameters)\u001b[0m\n\u001b[1;32m    607\u001b[0m \u001b[38;5;250m\u001b[39m\u001b[38;5;124;03m\"\"\"\u001b[39;00m\n\u001b[1;32m    608\u001b[0m \u001b[38;5;124;03mExecute a query.\u001b[39;00m\n\u001b[1;32m    609\u001b[0m \n\u001b[0;32m   (...)\u001b[0m\n\u001b[1;32m    619\u001b[0m \u001b[38;5;124;03m    parameters, which will each be bound in turn).\u001b[39;00m\n\u001b[1;32m    620\u001b[0m \u001b[38;5;124;03m\"\"\"\u001b[39;00m\n\u001b[0;32m--> 621\u001b[0m \u001b[38;5;28;43mself\u001b[39;49m\u001b[38;5;241;43m.\u001b[39;49m\u001b[43m_prepare_execute\u001b[49m\u001b[43m(\u001b[49m\u001b[43moperation\u001b[49m\u001b[43m,\u001b[49m\u001b[43m \u001b[49m\u001b[43mparameters\u001b[49m\u001b[43m)\u001b[49m\n\u001b[1;32m    622\u001b[0m handle, \u001b[38;5;28mself\u001b[39m\u001b[38;5;241m.\u001b[39m_rowcount \u001b[38;5;241m=\u001b[39m \u001b[38;5;28mself\u001b[39m\u001b[38;5;241m.\u001b[39m_stmt\u001b[38;5;241m.\u001b[39mexecute_query()\n\nFile \u001b[0;32m/opt/homebrew/lib/python3.11/site-packages/adbc_driver_manager/dbapi.py:590\u001b[0m, in \u001b[0;36mCursor._prepare_execute\u001b[0;34m(self, operation, parameters)\u001b[0m\n\u001b[1;32m    589\u001b[0m \u001b[38;5;28;01mtry\u001b[39;00m:\n\u001b[0;32m--> 590\u001b[0m     \u001b[38;5;28;43mself\u001b[39;49m\u001b[38;5;241;43m.\u001b[39;49m\u001b[43m_stmt\u001b[49m\u001b[38;5;241;43m.\u001b[39;49m\u001b[43mprepare\u001b[49m\u001b[43m(\u001b[49m\u001b[43m)\u001b[49m\n\u001b[1;32m    591\u001b[0m \u001b[38;5;28;01mexcept\u001b[39;00m NotSupportedError:\n\u001b[1;32m    592\u001b[0m     \u001b[38;5;66;03m# Not all drivers support it\u001b[39;00m\n\nFile \u001b[0;32m/opt/homebrew/lib/python3.11/site-packages/adbc_driver_manager/_lib.pyx:1108\u001b[0m, in \u001b[0;36madbc_driver_manager._lib.AdbcStatement.prepare\u001b[0;34m()\u001b[0m\n\nFile \u001b[0;32m/opt/homebrew/lib/python3.11/site-packages/adbc_driver_manager/_lib.pyx:399\u001b[0m, in \u001b[0;36madbc_driver_manager._lib.check_error\u001b[0;34m()\u001b[0m\n\n\u001b[0;31mOperationalError\u001b[0m: UNKNOWN: [FlightSQL] [FlightSQL]  (Unknown; Prepare)\n\nThe above exception was the direct cause of the following exception:\n\n\u001b[0;31mDatabaseError\u001b[0m                             Traceback (most recent call last)\nCell \u001b[0;32mIn[4], line 1\u001b[0m\n\u001b[0;32m----> 1\u001b[0m \u001b[43mpd\u001b[49m\u001b[38;5;241;43m.\u001b[39;49m\u001b[43mread_sql\u001b[49m\u001b[43m(\u001b[49m\n\u001b[1;32m      2\u001b[0m \u001b[43m    \u001b[49m\u001b[38;5;124;43m'\u001b[39;49m\u001b[38;5;124;43mselect * from Samples.\u001b[39;49m\u001b[38;5;124;43m\"\u001b[39;49m\u001b[38;5;124;43msamples.dremio.com\u001b[39;49m\u001b[38;5;124;43m\"\u001b[39;49m\u001b[38;5;124;43m.\u001b[39;49m\u001b[38;5;124;43m\"\u001b[39;49m\u001b[38;5;124;43mNYC-taxi-trips-iceberg\u001b[39;49m\u001b[38;5;124;43m\"\u001b[39;49m\u001b[38;5;124;43m limit 10\u001b[39;49m\u001b[38;5;124;43m'\u001b[39;49m\u001b[43m,\u001b[49m\u001b[43m \u001b[49m\u001b[43mconn\u001b[49m\u001b[43m)\u001b[49m\n\nFile \u001b[0;32m/opt/homebrew/lib/python3.11/site-packages/pandas/io/sql.py:654\u001b[0m, in \u001b[0;36mread_sql\u001b[0;34m(sql, con, index_col, coerce_float, params, parse_dates, columns, chunksize, dtype_backend, dtype)\u001b[0m\n\u001b[1;32m    652\u001b[0m \u001b[38;5;28;01mwith\u001b[39;00m pandasSQL_builder(con) \u001b[38;5;28;01mas\u001b[39;00m pandas_sql:\n\u001b[1;32m    653\u001b[0m     \u001b[38;5;28;01mif\u001b[39;00m \u001b[38;5;28misinstance\u001b[39m(pandas_sql, SQLiteDatabase):\n\u001b[0;32m--> 654\u001b[0m         \u001b[38;5;28;01mreturn\u001b[39;00m \u001b[43mpandas_sql\u001b[49m\u001b[38;5;241;43m.\u001b[39;49m\u001b[43mread_query\u001b[49m\u001b[43m(\u001b[49m\n\u001b[1;32m    655\u001b[0m \u001b[43m            \u001b[49m\u001b[43msql\u001b[49m\u001b[43m,\u001b[49m\n\u001b[1;32m    656\u001b[0m \u001b[43m            \u001b[49m\u001b[43mindex_col\u001b[49m\u001b[38;5;241;43m=\u001b[39;49m\u001b[43mindex_col\u001b[49m\u001b[43m,\u001b[49m\n\u001b[1;32m    657\u001b[0m \u001b[43m            \u001b[49m\u001b[43mparams\u001b[49m\u001b[38;5;241;43m=\u001b[39;49m\u001b[43mparams\u001b[49m\u001b[43m,\u001b[49m\n\u001b[1;32m    658\u001b[0m \u001b[43m            \u001b[49m\u001b[43mcoerce_float\u001b[49m\u001b[38;5;241;43m=\u001b[39;49m\u001b[43mcoerce_float\u001b[49m\u001b[43m,\u001b[49m\n\u001b[1;32m    659\u001b[0m \u001b[43m            \u001b[49m\u001b[43mparse_dates\u001b[49m\u001b[38;5;241;43m=\u001b[39;49m\u001b[43mparse_dates\u001b[49m\u001b[43m,\u001b[49m\n\u001b[1;32m    660\u001b[0m \u001b[43m            \u001b[49m\u001b[43mchunksize\u001b[49m\u001b[38;5;241;43m=\u001b[39;49m\u001b[43mchunksize\u001b[49m\u001b[43m,\u001b[49m\n\u001b[1;32m    661\u001b[0m \u001b[43m            \u001b[49m\u001b[43mdtype_backend\u001b[49m\u001b[38;5;241;43m=\u001b[39;49m\u001b[43mdtype_backend\u001b[49m\u001b[43m,\u001b[49m\n\u001b[1;32m    662\u001b[0m \u001b[43m            \u001b[49m\u001b[43mdtype\u001b[49m\u001b[38;5;241;43m=\u001b[39;49m\u001b[43mdtype\u001b[49m\u001b[43m,\u001b[49m\n\u001b[1;32m    663\u001b[0m \u001b[43m        \u001b[49m\u001b[43m)\u001b[49m\n\u001b[1;32m    665\u001b[0m     \u001b[38;5;28;01mtry\u001b[39;00m:\n\u001b[1;32m    666\u001b[0m         _is_table_name \u001b[38;5;241m=\u001b[39m pandas_sql\u001b[38;5;241m.\u001b[39mhas_table(sql)\n\nFile \u001b[0;32m/opt/homebrew/lib/python3.11/site-packages/pandas/io/sql.py:2330\u001b[0m, in \u001b[0;36mSQLiteDatabase.read_query\u001b[0;34m(self, sql, index_col, coerce_float, parse_dates, params, chunksize, dtype, dtype_backend)\u001b[0m\n\u001b[1;32m   2319\u001b[0m \u001b[38;5;28;01mdef\u001b[39;00m \u001b[38;5;21mread_query\u001b[39m(\n\u001b[1;32m   2320\u001b[0m     \u001b[38;5;28mself\u001b[39m,\n\u001b[1;32m   2321\u001b[0m     sql,\n\u001b[0;32m   (...)\u001b[0m\n\u001b[1;32m   2328\u001b[0m     dtype_backend: DtypeBackend \u001b[38;5;241m|\u001b[39m Literal[\u001b[38;5;124m\"\u001b[39m\u001b[38;5;124mnumpy\u001b[39m\u001b[38;5;124m\"\u001b[39m] \u001b[38;5;241m=\u001b[39m \u001b[38;5;124m\"\u001b[39m\u001b[38;5;124mnumpy\u001b[39m\u001b[38;5;124m\"\u001b[39m,\n\u001b[1;32m   2329\u001b[0m ) \u001b[38;5;241m-\u001b[39m\u001b[38;5;241m>\u001b[39m DataFrame \u001b[38;5;241m|\u001b[39m Iterator[DataFrame]:\n\u001b[0;32m-> 2330\u001b[0m     cursor \u001b[38;5;241m=\u001b[39m \u001b[38;5;28;43mself\u001b[39;49m\u001b[38;5;241;43m.\u001b[39;49m\u001b[43mexecute\u001b[49m\u001b[43m(\u001b[49m\u001b[43msql\u001b[49m\u001b[43m,\u001b[49m\u001b[43m \u001b[49m\u001b[43mparams\u001b[49m\u001b[43m)\u001b[49m\n\u001b[1;32m   2331\u001b[0m     columns \u001b[38;5;241m=\u001b[39m [col_desc[\u001b[38;5;241m0\u001b[39m] \u001b[38;5;28;01mfor\u001b[39;00m col_desc \u001b[38;5;129;01min\u001b[39;00m cursor\u001b[38;5;241m.\u001b[39mdescription]\n\u001b[1;32m   2333\u001b[0m     \u001b[38;5;28;01mif\u001b[39;00m chunksize \u001b[38;5;129;01mis\u001b[39;00m \u001b[38;5;129;01mnot\u001b[39;00m \u001b[38;5;28;01mNone\u001b[39;00m:\n\nFile \u001b[0;32m/opt/homebrew/lib/python3.11/site-packages/pandas/io/sql.py:2278\u001b[0m, in \u001b[0;36mSQLiteDatabase.execute\u001b[0;34m(self, sql, params)\u001b[0m\n\u001b[1;32m   2275\u001b[0m     \u001b[38;5;28;01mraise\u001b[39;00m ex \u001b[38;5;28;01mfrom\u001b[39;00m \u001b[38;5;21;01minner_exc\u001b[39;00m\n\u001b[1;32m   2277\u001b[0m ex \u001b[38;5;241m=\u001b[39m DatabaseError(\u001b[38;5;124mf\u001b[39m\u001b[38;5;124m\"\u001b[39m\u001b[38;5;124mExecution failed on sql \u001b[39m\u001b[38;5;124m'\u001b[39m\u001b[38;5;132;01m{\u001b[39;00msql\u001b[38;5;132;01m}\u001b[39;00m\u001b[38;5;124m'\u001b[39m\u001b[38;5;124m: \u001b[39m\u001b[38;5;132;01m{\u001b[39;00mexc\u001b[38;5;132;01m}\u001b[39;00m\u001b[38;5;124m\"\u001b[39m)\n\u001b[0;32m-> 2278\u001b[0m \u001b[38;5;28;01mraise\u001b[39;00m ex \u001b[38;5;28;01mfrom\u001b[39;00m \u001b[38;5;21;01mexc\u001b[39;00m\n\n\u001b[0;31mDatabaseError\u001b[0m: Execution failed on sql 'select * from Samples.\"samples.dremio.com\".\"NYC-taxi-trips-iceberg\" limit 10': UNKNOWN: [FlightSQL] [FlightSQL]  (Unknown; Prepare)"
}["stack"])

---------------------------------------------------------------------------
OperationalError                          Traceback (most recent call last)
File /opt/homebrew/lib/python3.11/site-packages/pandas/io/sql.py:2266, in SQLiteDatabase.execute(self, sql, params)
   2265 try:
-> 2266     cur.execute(sql, *args)
   2267     return cur

File /opt/homebrew/lib/python3.11/site-packages/adbc_driver_manager/dbapi.py:621, in Cursor.execute(self, operation, parameters)
    607 """
    608 Execute a query.
    609 
   (...)
    619     parameters, which will each be bound in turn).
    620 """
--> 621 self._prepare_execute(operation, parameters)
    622 handle, self._rowcount = self._stmt.execute_query()

File /opt/homebrew/lib/python3.11/site-packages/adbc_driver_manager/dbapi.py:590, in Cursor._prepare_execute(self, operation, parameters)
    589 try:
--> 590     self._stmt.prepare()
    591 except NotSupportedError:
    592     # Not all drivers support it

File /opt/homebrew/lib/python3